In [1]:
#imports
import os
import shutil
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import random
import Augmentor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from tqdm import tqdm

2025-03-11 20:44:19.342186: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-11 20:44:19.485659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741697059.539501    4798 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741697059.556320    4798 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 20:44:19.688244: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [5]:
dataPath = "/home/b1az3/code/ml/trainingData"

In [2]:
# import images
#--- words.txt ---------------------------------------------------------------#
#
# iam database word information
#
# format: a01-000u-00-00 ok 154 1 408 768 27 51 AT A
#
#     a01-000u-00-00  -> word id for line 00 in form a01-000u
#     ok              -> result of word segmentation
#                            ok: word was correctly
#                            er: segmentation of word can be bad
#
#     154             -> graylevel to binarize the line containing this word
#     1               -> number of components for this word NOTE: dont think this exists idk why
#     408 768 27 51   -> bounding box around this word in x,y,w,h format
#     AT              -> the grammatical tag for this word, see the
#                        file tagset.txt for an explanation
#     A               -> the transcription for this word
#
class img:
    def __init__ (this,letter,p1,p2):
        this.letter = letter
        this.inPath = p1
        this.savePath = p2
    
images = []

In [16]:
#import directory for dataset 1 - IAM



In [3]:
#To add a pixel of border around each picture, because zooming out uses the edge pixels
def addBorder (path):
    t = cv2.imread(dataPath+path+".png")
    pic = np.array(t)
    x,y,z = pic.shape #ALSO change colour here if needed
    
    column = np.array([[[255,255,255]] for i in range (x)]) # to add 1 pixel of white around the picture, because the picture is cropped to exactly where the side contains black and zoom function does not take that well
    row = np.array([[[255,255,255] for i in range (y+2)]])
    pic = np.concatenate([column,pic,column], axis=1)
    pic = np.concatenate([row,pic,row],axis=0)
    cv2.imwrite(dataPath+path+".png",pic)
    return pic


In [6]:
#import directory for dataset 1 - EMNIST
images = []
directory = open(dataPath + "/ascii/words.txt","r").read().splitlines()
len(directory)
directory = directory[18:] #first 18 lines are instructions
for i in directory:
    data = i.split(" ")
    a = data[0].split("-")
    loc = "/"+a[0]+"/"+a[0]+"-"+a[1]+"/"+data[0]
    letter = data[-1]
    if len(letter) == 1:
        images.append(img(letter,"/words"+loc,"/IAM"+loc))
print(len(images))

if open("log.txt","r").read() == "":
    open("log.txt","w").write("added border")
    for img_ in images:
        try:
            addBorder(img_.inPath)
        except:
            print(dataPath+img_.inPath+".png")


dir = os.listdir(dataPath + "/by_class/")
for i in dir:
    letter = chr(int("0x"+i,0))
    subDir = os.listdir(dataPath + "/by_class/"+i)
    for j in subDir:
        if j[-4:] != ".mit":
            imgDir = os.listdir(dataPath + "/by_class/"+i+"/"+j)
            for k in imgDir:
                images.append(img(letter,"/by_class/"+i+"/"+j+"/"+k[:-4],"/EMNIST/"+i+"/"+j+"/"+k[:-4]))
print (len(images))

18079
1960591


In [19]:
for i in range (100):
    print (images[i].inPath,letter,images[i].savePath)

/words/a01/a01-000u/a01-000u-00-00 D /IAM/a01/a01-000u/a01-000u-00-00
/words/a01/a01-000u/a01-000u-02-05 D /IAM/a01/a01-000u/a01-000u-02-05
/words/a01/a01-000u/a01-000u-03-02 D /IAM/a01/a01-000u/a01-000u-03-02
/words/a01/a01-000u/a01-000u-04-02 D /IAM/a01/a01-000u/a01-000u-04-02
/words/a01/a01-000u/a01-000u-06-01 D /IAM/a01/a01-000u/a01-000u-06-01
/words/a01/a01-000u/a01-000u-06-02 D /IAM/a01/a01-000u/a01-000u-06-02
/words/a01/a01-000u/a01-000u-06-06 D /IAM/a01/a01-000u/a01-000u-06-06
/words/a01/a01-000x/a01-000x-00-00 D /IAM/a01/a01-000x/a01-000x-00-00
/words/a01/a01-000x/a01-000x-01-10 D /IAM/a01/a01-000x/a01-000x-01-10
/words/a01/a01-000x/a01-000x-02-05 D /IAM/a01/a01-000x/a01-000x-02-05
/words/a01/a01-000x/a01-000x-03-04 D /IAM/a01/a01-000x/a01-000x-03-04
/words/a01/a01-000x/a01-000x-04-10 D /IAM/a01/a01-000x/a01-000x-04-10
/words/a01/a01-000x/a01-000x-05-00 D /IAM/a01/a01-000x/a01-000x-05-00
/words/a01/a01-000x/a01-000x-05-04 D /IAM/a01/a01-000x/a01-000x-05-04
/words/a01/a01-003/a

In [20]:
def augment (path,savePath):
    # Converting the input sample image to an array
    # Reshaping the input image
    datagen = ImageDataGenerator(zoom_range = [1,2],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1)
   
    # Loading a sample image 
    img = load_img(path) 
    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape) 
    
    # Generating and saving 5 augmented samples 
    # using the above defined parameters. 
    i = 0
    if not os.path.exists(savePath):
        os.makedirs(savePath)
    else:
        if len(os.listdir(savePath)) > 0:
            return
    for batch in datagen.flow(x, batch_size = 1,
                            save_to_dir =savePath, 
                            save_prefix ='image', save_format ='jpeg'):
        i += 1
        if i >= 5:
            break

In [21]:
#execute augment function, will take long time
#change
last = ""
total = len(images)
last = 0
imagesCopy = images.copy()
count = len(open("progress.txt","r").read())
print(count)
progress = open("progress.txt","a")
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    count +=1
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)
    progress.write(".")
progress.close()
open("progress.txt","w").close()    

16384


100%|██████████| 1944207/1944207 [17:51<00:00, 1814.58it/s]


In [22]:
print(len(images))

1960591


In [27]:

count = len(open("progress2.txt","r").read())+1

print (count)
progress = open("progress2.txt","a")

def resize(image_path,size,savePath): #resize to size and add black padding around unknown
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_pad(image, size[0],size[1])
    tf.keras.utils.save_img(savePath, image, file_format="jpeg")
    os.remove(image_path)
total = len(images)
last = int(count/total*1000)
progressBar = ""

progressBar = tqdm(range(count,len(images)))
print(progressBar)
def recurSearch (dir,savedir):
    files = os.listdir(dir)
    if files[0][-4:] == "jpeg":
        print (dir,end="\r")
        for file in files:
            if not os.path.exists(savedir):
                os.makedirs(savedir)
            if len(os.listdir(savedir)) < 5:
                resize(dir+"/"+file,(224,224),savedir+"/"+file[:-5])
            progress.write(".")
        progressBar
    else:
        for file in files:
            recurSearch(dir+"/"+file,savedir+"/"+file)
    os.rmdir(dir)


recurSearch(dataPath+"/augmentedWords",dataPath+"/final_aug_words")
progress.close()
open("progress2.txt","w").close()
    


0it [00:54, ?it/s]


7240540


0it [00:00, ?it/s]


In [ ]:
import shutil
shutil.rmtree(dataPath+"/augmentedWords")